In [1]:
from flexanomalies.models import AutoEncoder
from flexanomalies.utils.load_data import split_data, federate_data
from flexanomalies.datasets.preprocessing_utils import (
    create_windows,
    encode_and_bind,
    scaling,
    impute_lost_values,
)
from flexanomalies.utils.metrics import print_metrics
from flexanomalies.utils.process_scores import (
    process_scores_with_percentile,
    process_scores_with_threshold,
)
from sklearn.preprocessing import StandardScaler
from flexanomalies.pool.primitives_deepmodel import (
    build_server_model_ae,
    copy_model_to_clients_ae,
    train_ae,
    set_aggregated_weights_ae,
    weights_collector_ae,
    evaluate_global_model,
    evaluate_global_model_clients,
    threshold_collector_ae,
)
from flexanomalies.pool.aggregators_favg import aggregate_ae
from flexanomalies.utils.save_results import save_experiments_results
from flex.pool import FlexPool
from flexanomalies.utils.metrics import *
import pandas as pd
import numpy as np

2024-01-08 15:07:35.326528: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-08 15:07:35.568763: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-08 15:07:35.597230: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-01-08 15:07:35.597247: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [2]:
file_path = "../flex-anomalies/flexanomalies/datasets/data/corrected.gz"
split_test = 0.3

df = pd.read_csv(file_path, header=None)

# process labels
df.loc[df[41] != "normal.", 41] = 1
df.loc[df[41] == "normal.", 41] = 0
labels = df[41]
df = df.drop([41], axis=1)

features_to_encode = [1, 2, 3]
df = df.drop(features_to_encode, axis=1)


In [3]:
model_params = {
    "epochs": 50,
    "input_dim": df.shape[1],
    "batch_size": 32,
    "neurons": [8, 4, 8],
    "hidden_act": ["relu", "relu", "relu"],
    "preprocess":False,
    "w_size": 30,
    "n_pred": 10,
    "contamination": 0.1,
}

In [4]:
X = scaling(np.array(df.iloc[:, :].astype(float)))
y = np.array(labels)
X_train, X_test, l_train, l_test = split_data(X, y, split_size=0.30)

In [5]:
def create_windows(w_size, n_pred, X_train, X_test, l_train, l_test):
    X_train_windows = []
    y_train_windows = []
    X_test_windows = []
    y_test_windows = []
   

    for i in range(0, len(X_train), n_pred):
        temp_xtrain = X_train[i : w_size + i, :]
        temp_ytrain = l_train[i : w_size + i]
        if len(temp_xtrain) < w_size or len(temp_ytrain) < n_pred:
            break
        X_train_windows.append(temp_xtrain)

        y_train_windows.append(temp_ytrain)

    for i in range(0, len(X_test), n_pred):

        temp_xtest = X_test[i : w_size + i, :]
        temp_ytest = l_test[i : w_size + i]
        if (
            len(temp_xtest) < w_size
            or len(temp_ytest) < n_pred
            
        ):
            break

        X_test_windows.append(temp_xtest)

        y_test_windows.append(temp_ytest)
        
    return (
        np.array(X_train_windows),
        np.array(y_train_windows),
        np.array(X_test_windows),
        np.array(y_test_windows),
        )


In [6]:
(
    X_train_windows,
    y_train_windows,
    X_test_windows,
    y_test_windows,
    
) = create_windows(model_params["w_size"], model_params["n_pred"], X_train, X_test,l_train, l_test)

print("X_train shape == {}.".format(np.array(X_train_windows).shape))
print("y_train shape == {}.".format(np.array(y_train_windows).shape))
print("X_test shape == {}.".format(np.array(X_test_windows).shape))
print("y_test shape == {}.".format(np.array(y_test_windows).shape))




X_train shape == (21770, 30, 38).
y_train shape == (21770, 30).
X_test shape == (9328, 30, 38).
y_test shape == (9328, 30).


In [7]:
model = AutoEncoder(**model_params)


2024-01-08 15:07:45.008470: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-01-08 15:07:45.008848: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2024-01-08 15:07:45.008861: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (beatriz-IdeaPad-3-15ITL6): /proc/driver/nvidia/version does not exist
2024-01-08 15:07:45.009080: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
flex_dataset = federate_data(5, X_train_windows,y_train_windows)
pool = FlexPool.client_server_pool(
    fed_dataset=flex_dataset,
    server_id="autoencoder_server",
    init_func=build_server_model_ae,
    model=model,
)

In [10]:
for i in range(3):
    print(f"\nRunning round: {i}\n")
    pool.servers.map(copy_model_to_clients_ae, pool.clients)
    pool.clients.map(train_ae)
    pool.aggregators.map(weights_collector_ae, pool.clients)
    pool.aggregators.map(aggregate_ae)
    pool.aggregators.map(set_aggregated_weights_ae, pool.servers)
output_model = pool.servers._models["autoencoder_server"]["model"]



Running round: 0

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......dense
.........vars
............0
............1
......dense_1
.........vars
............0
............1
......dense_2
.........vars
............0
............1
......dense_3
.........vars
............0
............1
...optimizer
......vars
.........0
...vars
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2024-01-08 15:07:50         2233
variables.h5                                   2024-01-08 15:07:50        19728
metadata.json                                  2024-01-08 15:07:50           64
Keras model archive loading:
File Name                                             Modified             Size
config.json                                    2024-01-08 15:07:50         2233
variables.h5                                   2024-01-08 15:07:50        19728
metadata.json          

In [ ]:
def evaluate_global_model(
    model,
    X,
    y,
    metrics=["Accuracy", "Precision", "F1", "Recall", "AUC_ROC"],
    threshold=None,
):
    prediction = model.model.predict(X)
    print(prediction.shape)
    print(y.shape)
    print(np.mean((X - prediction), axis=2).shape)

    d_scores = np.mean((X - prediction), axis=2).flatten()
    if threshold is None:
        threshold = process_scores_with_percentile(d_scores, 0.1)
        print(threshold)

    l = (d_scores > threshold).astype("int").ravel()
    model.result_metrics_ = print_metrics(metrics,y, l)